0. 외부함수 사용을 위해 필요한 라이브러리 불러오기

In [ ]:
import finterstellar as fs
import pandas as pd
import numpy as np

In [ ]:
K10_dict = {
    '005930' : '삼성전자',
    '000660' : 'SK하이닉스',
    '051910' : 'LG화학',
    '068270' : '셀트리온',
    '005380' : '현대차',
    '207940' : '삼성바이오로직스',
    '005490' : 'POSCO',
    '028260' : '삼성물산',
    '015760' : '한국전력',
    '035420' : 'NAVER'
}

1. 시가총액이 있는 엑셀 파일 읽기: read_excel()

In [ ]:
path = './data/'     # 파일위치
n = 'K10_info.xlsx'     # 파일명

In [ ]:
file_name = path + n    # 읽어올 파일위치+파일명 지정
stock_info = pd.read_excel(file_name, converters={'code':str})
'''
    pd.read_excel(파일명) - 엑셀파일을 읽어 데이터프레임으로 변환시켜 돌려주는 함수
    converters={'code':str} - 데이터 형식을 '문자'형식으로 지정, 
    생략 시 pd가 이 컬럼을 자동으로 숫자로 인식해 종목코드 앞쪽 0을 삭제해버리게됨
'''
stock_info.set_index('code', inplace=True)
stock_info.head(10)

4. 10개 종목의 시세 데이터 로딩 : create_portfolio_df()

In [ ]:
n = 'K10_prices.xlsx'     # 파일명

In [ ]:
file_name = path + n    # 읽어올 파일위치+파일명 지정
price = pd.read_excel(file_name)
'''
    pd.read_excel(파일명) - 엑셀파일을 읽어 데이터프레임으로 변환시켜 돌려주는 함수
    converters={'code':str} - 데이터 형식을 '문자'형식으로 지정, 
    생략 시 pd가 이 컬럼을 자동으로 숫자로 인식해 종목코드 앞쪽 0을 삭제해버리게됨
'''
price.set_index('trade_date', inplace=True)
price.tail()

In [ ]:
ld = fs.LoadData()

In [ ]:
price = ld.date_formatting(price)
price.head()

In [ ]:
base_date = '2018-01-02'

In [ ]:
price = price[base_date:]
price.head()

In [ ]:
stock_info['outstanding']

In [ ]:
market_cap = pd.DataFrame()
market_cap = price * stock_info['outstanding']
market_cap.head()

In [ ]:
market_cap['K10'] = market_cap.sum(axis=1)
market_cap.tail()

In [ ]:
index = pd.DataFrame()
# 첫 날짜를 기준으로 지수화
index['K10'] = round( market_cap['K10'] / market_cap.loc[base_date, 'K10'], 4 ) * 100
# 지수 = 일자별 시가총액 / 기준일 시가총액 * 100 
index.head()

K10 지수 그래프 그리기

In [ ]:
v = fs.Visualize()

In [ ]:
v.price_view(index, base_date, ['K10'])
# price_view(데이터프레임, 기준일자, [종목코드], (사이즈)) - 챠트 그리기

KOSPI200과 비교

In [ ]:
K200 = ld.make_historical_price_df(path, 'KOSPI 200')

In [ ]:
K200.head()

In [ ]:
index['K200'] = K200.loc[base_date:]
# 1월2일 이후 지수만 idx의 K200 컬럼에 담음
index.head()

In [ ]:
index = index.fillna(method='ffill')
# NaN 값을 앞의 값으로 채움

In [ ]:
v.index_view(index, base_date, ['K200', 'K10'])
# index_view(데이터프레임, 기준일자, [종목코드], (사이즈)) - 비교를 위해 인덱스화한 그래프 그리기

## 펀드 운용

In [ ]:
# 펀드 기본정보 세팅
CU = 1000     # 설정/환매 단위
base_date = pd.to_datetime('2018-01-02')     # 설정 기준일
fund_shares = 0     # 최초 설정수량
fund_price = 10000    # 최초 기준가
interest_rate = 0.02     # 이자율 (현금보유액에 사용)
stock_holdings = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
cash_value = 0

In [ ]:
# 주식운용에 필요한 각종 정보를 담을 데이터프레임 생성
K10_Fund = pd.DataFrame(columns=['Price', 'NAV', 'Shares', 'Stock_value', 'Cash_value'])    
# 일별 NAV, 주식보유액, 현금보유액
K10_Fund

In [ ]:
K10_weights = pd.DataFrame(columns=market_cap.columns[:-1])     
# K10 각 종목의 보유비중을 담을 데이터프레임 생성
K10_weights

In [ ]:
K10_holdings = pd.DataFrame(columns=market_cap.columns[:-1])     
# K10 각 종목의 보유비중을 담을 데이터프레임 생성
K10_holdings

In [ ]:
stock_holdings

In [ ]:
d = base_date

In [ ]:
# 1. 기존 보유잔고를 기준으로 기준 NAV 계산 : 종목별 잔고 * 종목별 현재가 + 현금
stock_prices = price.loc[d]
stock_value = sum(stock_holdings * stock_prices)     # 주식 잔고 = 보유수량 * 주가
NAV_old = stock_value + cash_value     # 전체 잔고 = 주식잔고 + 현금잔고
print(stock_prices)
print(NAV_old)

In [ ]:
# 2. Creation, Redemption을 반영해 펀드잔고 조정 : 변경좌수
def creation_redemption(v):     # 설정/환매
    creation = np.random.randint(0, 5) * CU     # 랜덤하게 설정/환매 수량 산출
    if v>10000:   # 1만좌 이상일 경우만 해지 가능 (수량이 -가 되지 않도록)
        redemption = np.random.randint(0, 5) * CU
    else:
        redemption = 0
    change = creation - redemption     # 총 좌수 = 기존좌수 + 설정좌수 - 환매좌수
    return(change)

In [ ]:
# 3. 기준가 산출 : NAV / 펀드좌수
if d > base_date:
    base_price = NAV_old / fund_shares    # 기준가격 = 펀드총자산 / 펀드총좌수
    share_chg = creation_redemption(fund_shares)
else:
    base_price = fund_price  
    share_chg = 10000  
print(base_price)

In [ ]:
# 4. 펀드좌수를 반영해 NAV 조정 : 신규 NAV = 기준 NAV + (기준가 * 변경좌수)
# 총 펀드 좌수에 반영
fund_shares = fund_shares + share_chg     # 펀드좌수 = 기존펀드좌수 + 변동좌수
# 펀드 입출금액 변동분
NAV_chg = base_price * share_chg     # NAV 변동분 = 기준가격 * 변동좌수
# 신규 NAV
NAV = NAV_old + NAV_chg
print(share_chg, fund_shares, NAV_chg, NAV)

In [ ]:
# 5. 종목별 현재가를 반영해 종목별 보유비중 산정 : 종목별 시가총액 / 지수 시가총액
stock_weights = market_cap.loc[d, K10_weights.columns] / market_cap.loc[d, 'K10']   
# (종목별시가총액/K10시가총액) 을 계산해 종목별 보유비중 산정
K10_weights.loc[d] = stock_weights    # 보유비중 기록
print(stock_weights)

In [ ]:
# 6. 신규 NAV를 반영해 종목별 보유해야 할 수(holdings) 산출 : NAV * 보유비중
# 종목별 보유수량 산정
stock_holdings_old = stock_holdings
stock_holdings = np.floor( NAV * stock_weights / stock_prices ) 
# 신규 주식 거래량 계산 : 
stock_trades = stock_holdings - stock_holdings_old
# 입출금액 변동분 * 주식비중 / 주가  (내림처리, 소수점단위 매매 불가)
print(stock_trades.tolist())
print(stock_holdings.tolist())

In [ ]:
# 7. 주식보유액, 현금보유액 등 산출 : NAV = 주식보유액 + 현금
K10_holdings.loc[d] = stock_holdings
# 현금 보유량 증가 (이자율 반영)
cash_holdings = np.floor( cash_value * np.exp(interest_rate/365) ) 
stock_value = sum(stock_holdings * stock_prices)     # 주식 잔고
cash_value = int(NAV - stock_value)     # 현금 잔고
print(stock_value)
print(cash_value)

In [ ]:
# 8. 결과 저장
K10_Fund.loc[d, ('Price', 'NAV', 'Shares', 'Stock_value', 'Cash_value')] = \
(base_price, NAV, fund_shares, stock_value, cash_value)

In [ ]:
K10_Fund

In [ ]:
price.index

In [ ]:
# 펀드 기본정보 세팅
CU = 1000     # 설정/환매 단위
base_date = pd.to_datetime('2018-01-02')     # 설정 기준일
fund_shares = 0     # 최초 설정수량
fund_price = 10000    # 최초 기준가
interest_rate = 0.02     # 이자율 (현금보유액에 사용)
stock_holdings = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
cash_value = 0
share_chg = 10000

for d in price.index:     # 일자별로 순환하며
    # 1. 기존 보유잔고를 기준으로 기준 NAV 계산 : 종목별 잔고 * 종목별 현재가 + 현금
    stock_prices = price.loc[d]
    stock_value = sum(stock_holdings * stock_prices)     # 주식 잔고 = 보유수량 * 주가
    NAV_old = stock_value + cash_value     # 전체 잔고 = 주식잔고 + 현금잔고
            
    # 2. creation_redemption 함수는 위에서 별도 구현
    
    # 3. 기준가 산출 : NAV / 펀드좌수
    if d == base_date:
        base_price = fund_price  
        share_chg = 10000
    else:
        base_price = NAV_old / fund_shares    # 기준가격 = 펀드총자산 / 펀드총좌수
        share_chg = share_chg = creation_redemption(fund_shares)     # 설정/환매 함수 호출

    # 4. 펀드좌수를 반영해 NAV 조정 : 신규 NAV = 기준 NAV + (기준가 * 변경좌수)
    # 총 펀드 좌수에 반영
    fund_shares = fund_shares + share_chg     # 펀드좌수 = 기존펀드좌수 + 변동좌수
    # 펀드 입출금액 변동분
    NAV_chg = base_price * share_chg     # NAV 변동분 = 기준가격 * 변동좌수
    # 신규 NAV
    NAV = NAV_old + NAV_chg
    
    #print('Date:', d.date(), ' Share:', fund_shares, ' Chages:', share_chg)

    # 5. 종목별 현재가를 반영해 종목별 보유비중 산정 : 종목별 시가총액 / 지수 시가총액
    stock_weights = market_cap.loc[d, K10_weights.columns] / market_cap.loc[d, 'K10']   
    # (종목별시가총액/K10시가총액) 을 계산해 종목별 보유비중 산정
    K10_weights.loc[d] = stock_weights    # 보유비중 기록
    
    # 6. 신규 NAV를 반영해 종목별 보유해야 할 수(holdings) 산출 : NAV * 보유비중
    # 종목별 보유수량 산정
    stock_holdings_old = stock_holdings
    stock_holdings = np.floor( NAV * stock_weights / stock_prices ) 
    # 신규 주식 거래량 계산 : 
    stock_trades = stock_holdings - stock_holdings_old
    # 입출금액 변동분 * 주식비중 / 주가  (내림처리, 소수점단위 매매 불가)
    
    # 7. 주식보유액, 현금보유액 등 산출 : NAV = 주식보유액 + 현금
    K10_holdings.loc[d] = stock_holdings
    # 현금 보유량 증가 (이자율 반영)
    cash_holdings = np.floor( cash_value * np.exp(interest_rate/365) ) 
    stock_value = sum(stock_holdings * stock_prices)     # 주식 잔고
    cash_value = int(NAV - stock_value)     # 현금 잔고

    # 8. 결과 저장
    K10_Fund.loc[d, ('Price', 'NAV', 'Shares', 'Stock_value', 'Cash_value')] =\
    (base_price, NAV, fund_shares, stock_value, cash_value)

In [ ]:
# 펀드 수익률 vs 지수 수익률
Earnings = pd.DataFrame()
Earnings['K10'] = ( index['K10']) / index['K10'][base_date] * 100   
Earnings['K200'] = ( index['K200']) / index['K200'][base_date] * 100
Earnings['Fund'] = ( K10_Fund['Price'])/ K10_Fund['Price'][base_date] * 100
Earnings.dropna(inplace=True)

In [ ]:
Earnings.head()

In [ ]:
v.index_view(Earnings, base_date, ['K10', 'K200', 'Fund'], size=(15,5))

In [ ]:
# 일간 수익률
Daily = pd.DataFrame()
Daily['K10'] = (index['K10'] - index['K10'].shift(1)) / index['K10'].shift(1) 
Daily['K200'] = (index['K200'] - index['K200'].shift(1)) / index['K200'].shift(1) 
Daily['Fund'] = (K10_Fund['Price'] - K10_Fund['Price'].shift(1)) / K10_Fund['Price'].shift(1)  
Daily.dropna(inplace=True)

In [ ]:
v.price_view(Daily, base_date, ['K10', 'K200', 'Fund'], size=(15,3))

In [ ]:
# KOSPI200 대비 일간 추적오차율 
Daily['TE200'] = Daily['Fund'] - Daily['K200']    # 추적오차율 = 펀드수익률 - 지수수익률

In [ ]:
v.price_view(Daily, base_date, 'TE200', size=(15,3))

In [ ]:
# K10 대비 일간 추적오차율 
Daily['TE10'] = Daily['Fund'] - Daily['K10']

In [ ]:
v.price_view(Daily, base_date, 'TE10', size=(15,3))